# Hands-on with the NextOcean API: A virtual tutorial
+ Author: Miguel Chapela Rivas, Deimos Space
+ Date: 13/09/2023

## Outline
1. Initialize connection and login
2. Get service list
3. Get Service information
4. Order service
5. Get order status
6. Download products

## 1- Initialize connection and login

In [1]:
# Import the required library
import request4EO
import yaml
import requests
import os 

# Initialize the connection with request4EO API
requestAPI = request4EO.requestServiceAPI()

# Load credentials from the 'credentials.yaml' file
credentials = yaml.safe_load(open('credentials.yaml', 'r'))

# Extract the username and password from the credentials file
username = credentials['username']
password = credentials['password']

# Use the extracted username and password to log in to the API
login = requestAPI.login(username, password)

## 2- Get service list


In [3]:
# Use function get_services_list with the object created before
# ('service_id ', 'service_name')
services_list = requestAPI.get_services_list()
services_list

{'FISHERIES_MONITORING_AND_SURVEILLANCE_(FMS)': 'FMS: Fisheries Monitoring and Surveillance',
 'FPE:_FISHING_FOOTPRINT': 'FPE: Fishing provenance and ecolabelling - Fishing Footprint',
 'FPE:_FISHING_TRIPS': 'FPE: Fishing provenance and ecolabelling - Fishing Trips',
 'FFI__FISH_FARM_IMPACTS___DRIFT_SIMULATION': 'FFI: Fish Farm Impacts - Drift Simulation',
 'SITE_RISK_ASSESSMENT__SRA____METOCEAN': 'SRA: Site Risk Assessment - MetOcean',
 'SRA__SITE_RISK_ASSESSMENT___OIL_SPILL_PROBABILITY': 'SRA: Site Risk Assessment - Oil Spill Probability',
 'MONITORING_AQUACULTURE_STRUCTURES__MAS_': 'MAS: Monitoring Aquaculture Structures',
 'CFA__POTENTIAL_FISHING_AREAS': 'CFA: Characterization of Fishing Areas - Potential Fishing Areas'}

## 3- Get Service information

In [4]:
# Use function get_services with the object created before
# Returns a JSON with all the information about the service 
#service_id = 'CFA__POTENTIAL_FISHING_AREAS'
service_id = 'FPE:_FISHING_TRIPS'
service_info = requestAPI.get_service(service_id)
service_info

{'identifier': 'FPE:_FISHING_TRIPS',
 'title': 'FPE: Fishing provenance and ecolabelling - Fishing Trips',
 'description': 'The FPE - Fishing Trips service maps the trajectory of fishing vessels discriminated by colour according to their activity status (fishing and non-fishing), based on Automatic Identification System (AIS) point datasets.',
 'fileName': 'FPE:_FISHING_TRIPS_20221011071546_application_service',
 'inputFileTypes': ['VESSEL_PATHS_AIS_NE_ATLANTIC'],
 'serviceType': 'Processing_Service',
 'serviceURI': 'https://data-processing.services4eo.com/wps/',
 'serviceSubtype': 'WPS_1.0',
 'serviceName': 'FISH_TRI',
 'serviceCategory': {'id': 16,
  'slug': 'fisheries',
  'name': 'Fisheries',
  'createdAt': '2023-02-15T16:31:11.462677',
  'updatedAt': '2023-02-15T16:31:11.462677',
  'parent': {'id': 4,
   'slug': 'marine-and-maritime',
   'name': 'Marine & Maritime',
   'thumbnail': ''}},
 'keywords': ['AIS', 'Fishing', 'Trips', 'Fish', 'Provenance', 'Ecolabeling'],
 'publishers': [

### Get the service period and spatial footprint

In [5]:
# Extract servicePeriod
print('Minimum start date')
print(service_info['servicePeriod']['serviceStart'])
print('Maximum end date')
print(service_info['servicePeriod']['serviceStop'])
print()

# Extract serviceFootprint
# The vector featrure can be visualized in https://wktmap.com/
print('The service footprint is: ')
print(service_info['serviceFootprint'])


Minimum start date
2012-01-01T01:00:00
Maximum end date
2018-12-31T01:00:00

The service footprint is: 
MULTIPOLYGON(((-29.941406249999986 27.163067787973674,-22.382812499999986 29.483875765537963,-14.999999999999986 27.94228443709624,-9.462890624999986 32.20005534819621,-6.430664062499987 33.82140647652505,-5.595703124999987 36.338393838228555,-7.045898437499987 37.393106949944496,-8.441162109374986 37.40128947428784,-8.298339843749986 38.03514606300014,-8.847656249999986 38.87716915873357,-8.803710937499986 39.58984849551525,-8.408203124999986 41.29125390044174,-8.847656249999986 42.768218140889324,-13.769531249999986 43.02576472850212,-17.988281249999986 43.2502353481404,-21.328124999999986 44.17140053246825,-23.437499999999986 48.76798524483268,-32.050781249999986 49.57245856275113,-33.632812499999986 46.63909468774348,-42.949218749999986 45.418726354605006,-42.773437499999986 32.497057800813295,-32.578124999999986 30.397789896897468,-29.941406249999986 27.163067787973674)))


### Get the service input parameters

In [6]:
# Extract input identifiers and values
input_identifiers = [(item['identifier'], item['title']) for item in service_info['input']]

# Print the input identifiers
if not input_identifiers:
    print('The service has no addditional inputs')
else:
    print('Additional inputs required: \n')    
    for identifier, title in input_identifiers:
        print(f"{identifier} -> {title}")


Additional inputs required: 

MMSI -> Maritime Mobile Service Identity (Select All for no filter or a single MMSI)
main_gear -> Fishing technique of the vessel
country -> Registration flag of the vessels
vessel_loa -> LOA (Vessel Length [meters])


In [ ]:
# Extract input identifiers and values
input_identifiers = [(item['identifier'], item['literalData']['value']) for item in service_info['input']]

# Print the input identifiers
if not input_identifiers:
    print('The service has no addditional inputs')
else:      
    for identifier, value in input_identifiers:
        print(f"Identifier: {identifier}")
        print(f"Value: {value}")


## 4- Order service

Function -> order_service(self,service_id, service_parameters) 

Need to provide a dictionary with the service_parameters. The format is the follwing
<pre>
service_parameters = {
    "aoi": "<area in WKT>",
    "beginAt": "<begin date in format YYYY-mm-ddTHH:MM>",
    "endAt": "<end date in format YYYY-mm-ddTHH:MM>",

}
    #Each service specific inputs should be formated in a list of dictionaries like this: 

    "inputs": [{
                    "identifier": "<parameter_identifier_1>",
                    "value": "<parameter_value_1>"
                    },
                {
                    "identifier":"<parameter_identifier_2>",
                    "value":"<parameter_value_2>"
                    }]
    }

</pre>

### Order CFA: Characterization of Fishing Areas - Potential Fishing Areas

In [ ]:
from datetime import datetime
import shapely.wkt

# Define a polygon in Well-Known Text (WKT) format representing an area of interest (AOI)
aoi_str ="POLYGON ((-11.052246 40.615198, -8.76709 40.548466, -9.404297 37.337467, -10.678711 37.529337, -11.052246 40.615198))"
# Load the WKT polygon into a Shapely geometry object
aoi = shapely.wkt.loads(aoi_str)
# Define starting and final dates using datetime objects
starting_date = datetime(2018,8,1)
final_date = datetime(2018,8,1)

# Create a dictionary of service parameters for the CFA service.
service_parameters_CFA = {
    "aoi": aoi.wkt, # AOI in WKT format
    "beginAt": starting_date.strftime("%Y-%m-%dT%H:%M:%S"),  # Start date and time formatted as a string
    "endAt": final_date.strftime("%Y-%m-%dT%H:%M:%S"), # End date and time formatted as a string
    "inputs": []  # List of input parameters (empty in this example)
    }

service_parameters_CFA    

In [ ]:
order_info_CFA = requestAPI.order_service('CFA__POTENTIAL_FISHING_AREAS', service_parameters_CFA)
order_info_CFA

### Order FPE: Fishing provenance and ecolabelling - Fishing Trips

In [ ]:
# Define a polygon in Well-Known Text (WKT) format representing an area of interest (AOI)
aoi_str ="POLYGON((-11.335880496111539 39.499469670842466,-10.061466433611539 39.499469670842466,-10.061466433611539 38.57789510908854,-11.335880496111539 38.57789510908854,-11.335880496111539 39.499469670842466))"
# Load the WKT polygon into a Shapely geometry object
aoi = shapely.wkt.loads(aoi_str)
# Define starting and final dates using datetime objects
starting_date = datetime(2018,8,1)
final_date = datetime(2018,8,15)

# Create a dictionary of service parameters for the FPE Fishing Footprint service.
service_parameters_FPE_FT = {
    "aoi": aoi.wkt, # AOI in WKT format
    "beginAt": starting_date.strftime("%Y-%m-%dT%H:%M:%S"), # Start date and time formatted as a string
    "endAt": final_date.strftime("%Y-%m-%dT%H:%M:%S"), # End date and time formatted as a string
    # List of input parameters
    "inputs": [     
                {
                    "identifier": "MMSI",
                    "value": "All"
                    },
                {
                    "identifier": "main_gear",
                    "value": "All"
                    },  
                 {
                    "identifier": "country",
                    "value": "All"
                    },        
                {
                    "identifier":"vessel_loa",
                    "value":"All"
                    }] 
    }

service_parameters_FPE_FT    

In [ ]:
order_info_FPE_FT = requestAPI.order_service('FPE:_FISHING_TRIPS', service_parameters_FPE_FT)
order_info_FPE_FT

## 5 - Get order status

In [ ]:
# Get the 'id' of the order from the order_info_CFA dictionary
order_id_CFA = order_info_CFA["id"]
# Use the obtained order_id_CFA to request additional information about the order
order_info_CFA = requestAPI.get_order(order_id_CFA)
order_info_CFA

In [ ]:
print('The order {order_id} status is {order_status}'.format(order_id= order_id_CFA, order_status =order_info_CFA['status']))

# 6 - Download products

In [7]:
# order_id=  606417384 #Trial order CFA finshied
# Use the API to get information about the order with ID order_id_CFA
order_id = order_info_CFA["id"]
# order_id = order_info_FPE_FT["id"]
# order_id = 606417384 #Trial order CFA finshied
# order_id = 606417402 #Trial order FPE Fishing Trips finshied
order_info = requestAPI.get_order(order_id)


# Check the status of the order
if order_info['status'] == 'FINISHED':
    # If the order is finished, inform the user that it's ready for download
    print('The order {order_id} is finished and is ready for download'.format(order_id=order_id))
    
    # Get the products associated with this finished order
    order_products = requestAPI.get_order_products(order_id)
else:
    # If the order is not finished, inform the user of its status and that it cannot be downloaded
    print('The order {order_id} status {order_status}, it cannot be downloaded'.format(order_id=order_id, order_status=order_info['status']))


The order 606417402 is finished and is ready for download


In [8]:
order_products = requestAPI.get_order_products(order_id)
order_products

[{'jobId': 606423912,
  'productIdentifier': 'AIS_20180701_000000_25272484_FISH_TRI_LINES',
  'type': 'wms-time-series',
  'layers': {'LINES': [{'title': '2018-07-01T00:00',
     'downloadURI': 'https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/AIS_20180701_000000_25272484_FISH_TRI_LINES',
     'geoServerURI': 'https://data-lake.services4eo.com/geoserver/wms/reflect',
     'dateTime': '2018-07-01T00:00',
     'startTime': '2018-07-01T00:00:00',
     'stopTime': '2018-07-01T00:00:00',
     'wmsParams': {'format': 'image/png',
      'layers': 'shp_output_workspace:AIS_20180701_000000_25272484_FISH_TRI_LINES',
      'styles': 'FISH_TRI_LINES,FISH_TRI_POINTS',
      'bbox': '38.57789510908854,-11.335880496111539,39.499469670842466,-10.061466433611539',
      'time': ''},
     'timestamps': ['2018-08-01T01:00:00Z',
      '2018-08-02T01:00:00Z',
      '2018-08-03T01:00:00Z',
      '2018-08-04T01:00:00Z',
      '2018-08-05T01:00:00Z',
      '2018-08-06T01:0

### Check the products we have to download

In [9]:
# Iterate through the order_products
for item in order_products:
    # Iterate through the layers in each order product
    for layer_name, layer_data in item['layers'].items():
        # Extract the download URI for the layer
        download_uri = layer_data[0]['downloadURI']
        
        # Print information about the layer and its download URI
        print(f"Layer: {layer_name}, Download URI: {download_uri}")

Layer: LINES, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/AIS_20180701_000000_25272484_FISH_TRI_LINES
Layer: POINTS, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/AIS_20180701_000000_67049242_FISH_TRI_POINTS


### Function download_products(folder, download_url)

This function is designed to download a file from a given URL and save it to the specified folder. It also includes error checking for HTTP status codes and prints a confirmation message upon successful download.

In [10]:
def download_products(folder, download_url):
    # Create the destination folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Extract the filename from the download URL
    filename = download_url.split("/")[-1]

    # Define the full path for the downloaded product
    downloaded_product = os.path.join(folder, f"{filename}.zip")

    # Send an HTTP GET request to the download URL
    with requests.get(download_url, stream=True, allow_redirects=True) as r:
        # Check for HTTP errors
        if r.status_code < 200 or r.status_code >= 300:
            print("HTTP ERROR %s - %s", r.status_code, r.text)

        # Write the received content to the local file
        with open(downloaded_product, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        # Print a confirmation message
        print('The product {product} has been downloaded into {folder}'.format(product=filename, folder=folder))
      

### Download the products available from the order

In [11]:
# Define the folder where you want to download the products
download_folder = 'OUTPUTS'

# Iterate through the order_products
for item in order_products:
    # Iterate through the layers in each order product
    for layer_name, layer_data in item['layers'].items():
        # Extract the download URI for the layer
        download_uri = layer_data[0]['downloadURI']
        
        # Print information about the layer and its download URI
        print(f"Layer: {layer_name}, Download URI: {download_uri}")
        
        # Call the download_products function to download the layer to the specified folder
        download_products(download_folder, download_uri)

Layer: LINES, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/AIS_20180701_000000_25272484_FISH_TRI_LINES
The product AIS_20180701_000000_25272484_FISH_TRI_LINES has been downloaded into OUTPUTS
Layer: POINTS, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/AIS_20180701_000000_67049242_FISH_TRI_POINTS
The product AIS_20180701_000000_67049242_FISH_TRI_POINTS has been downloaded into OUTPUTS
